In [32]:
import pandas as pd
import numpy as np

nf = pd.read_csv('https://github.com/finnjinju/netflix/raw/master/all-weeks-countries1.csv')
nf['show_title'] = nf['show_title'].str.lower()
nf['season_title'] = nf['season_title'].str.lower()
nf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225760 entries, 0 to 225759
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   country_name                225760 non-null  object
 1   country_iso2                225760 non-null  object
 2   week                        225760 non-null  object
 3   category                    225760 non-null  object
 4   weekly_rank                 225760 non-null  int64 
 5   show_title                  225760 non-null  object
 6   season_title                109197 non-null  object
 7   cumulative_weeks_in_top_10  225760 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 13.8+ MB


In [33]:
imdb = pd.read_csv('https://github.com/finnjinju/netflix/raw/master/data.tsv',sep='\t')

<ipython-input-33-db0b9d04449d>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv('https://github.com/finnjinju/netflix/raw/master/data.tsv',sep='\t')


In [34]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10267986 entries, 0 to 10267985
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 705.0+ MB


In [35]:
imdbNew = imdb.copy()
imdbNew = imdbNew.rename(columns={'primaryTitle':'show_title'})
imdbNew['genres'].replace({"\\N": np.NaN},inplace=True)
imdbNew['show_title'] = imdbNew['show_title'].str.lower()
imdbNew = imdbNew.drop_duplicates(subset=['show_title'])
imdbNew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4580908 entries, 0 to 10267985
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   show_title      object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 349.5+ MB


In [36]:
left = pd.merge(nf,imdbNew[['show_title','genres']], on='show_title', how='left')
# left['genres'].isnull().sum()
# left.sample(20)

In [37]:
left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225760 entries, 0 to 225759
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   country_name                225760 non-null  object
 1   country_iso2                225760 non-null  object
 2   week                        225760 non-null  object
 3   category                    225760 non-null  object
 4   weekly_rank                 225760 non-null  int64 
 5   show_title                  225760 non-null  object
 6   season_title                109197 non-null  object
 7   cumulative_weeks_in_top_10  225760 non-null  int64 
 8   genres                      208383 non-null  object
dtypes: int64(2), object(7)
memory usage: 17.2+ MB


In [38]:
nanmask = left['genres'].isna()
leftmask = left[nanmask].copy()
leftmask.info()
leftmask.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17377 entries, 19 to 225752
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   country_name                17377 non-null  object
 1   country_iso2                17377 non-null  object
 2   week                        17377 non-null  object
 3   category                    17377 non-null  object
 4   weekly_rank                 17377 non-null  int64 
 5   show_title                  17377 non-null  object
 6   season_title                8498 non-null   object
 7   cumulative_weeks_in_top_10  17377 non-null  int64 
 8   genres                      0 non-null      object
dtypes: int64(2), object(7)
memory usage: 1.3+ MB


,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,genres
19,Argentina,AR,2023-10-22,TV,10,learning and fun with jose comelon,NaN,1,NaN
77,Argentina,AR,2023-10-01,TV,8,encounters,encounters: season 1,1,NaN
118,Argentina,AR,2023-09-17,TV,9,gangs of oslo,gangs of oslo: season 1,1,NaN
137,Argentina,AR,2023-09-10,TV,8,predators,predators: season 1,1,NaN
147,Argentina,AR,2023-09-03,Films,8,lion's den,NaN,1,NaN


In [39]:
imdb2 = imdb.copy()
imdb2['genres'].replace({"\\N": np.NaN},inplace=True)
imdb2.drop(columns=['primaryTitle'],inplace=True)
imdb2.rename(columns={'originalTitle':'show_title'},inplace=True)
imdb2['show_title'] = imdb2['show_title'].str.lower()
imdb2 = imdb2.drop_duplicates(subset=['show_title'])
imdb2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4602928 entries, 0 to 10267985
Data columns (total 8 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   show_title      object
 3   isAdult         object
 4   startYear       object
 5   endYear         object
 6   runtimeMinutes  object
 7   genres          object
dtypes: object(8)
memory usage: 316.1+ MB


In [40]:
remain = pd.merge(leftmask,imdb2[['show_title','genres']], on='show_title', how='left')
remain.info()
display(leftmask.head())
remain.set_index(leftmask.index, inplace=True)
remain.drop(columns=['genres_x'],inplace=True)
remain.rename(columns={'genres_y':'genres'},inplace=True)
display(remain.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17377 entries, 0 to 17376
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   country_name                17377 non-null  object
 1   country_iso2                17377 non-null  object
 2   week                        17377 non-null  object
 3   category                    17377 non-null  object
 4   weekly_rank                 17377 non-null  int64 
 5   show_title                  17377 non-null  object
 6   season_title                8498 non-null   object
 7   cumulative_weeks_in_top_10  17377 non-null  int64 
 8   genres_x                    0 non-null      object
 9   genres_y                    2101 non-null   object
dtypes: int64(2), object(8)
memory usage: 1.5+ MB


,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,genres
19,Argentina,AR,2023-10-22,TV,10,learning and fun with jose comelon,NaN,1,NaN
77,Argentina,AR,2023-10-01,TV,8,encounters,encounters: season 1,1,NaN
118,Argentina,AR,2023-09-17,TV,9,gangs of oslo,gangs of oslo: season 1,1,NaN
137,Argentina,AR,2023-09-10,TV,8,predators,predators: season 1,1,NaN
147,Argentina,AR,2023-09-03,Films,8,lion's den,NaN,1,NaN


,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,genres
19,Argentina,AR,2023-10-22,TV,10,learning and fun with jose comelon,NaN,1,NaN
77,Argentina,AR,2023-10-01,TV,8,encounters,encounters: season 1,1,"Drama,Horror,Thriller"
118,Argentina,AR,2023-09-17,TV,9,gangs of oslo,gangs of oslo: season 1,1,NaN
137,Argentina,AR,2023-09-10,TV,8,predators,predators: season 1,1,NaN
147,Argentina,AR,2023-09-03,Films,8,lion's den,NaN,1,NaN


In [41]:
left.loc[nanmask,'genres'] = remain.loc[nanmask,'genres']
left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225760 entries, 0 to 225759
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   country_name                225760 non-null  object
 1   country_iso2                225760 non-null  object
 2   week                        225760 non-null  object
 3   category                    225760 non-null  object
 4   weekly_rank                 225760 non-null  int64 
 5   show_title                  225760 non-null  object
 6   season_title                109197 non-null  object
 7   cumulative_weeks_in_top_10  225760 non-null  int64 
 8   genres                      210484 non-null  object
dtypes: int64(2), object(7)
memory usage: 25.3+ MB


In [42]:
imdbsearch = pd.read_csv('https://github.com/finnjinju/netflix/raw/master/remain.csv')
imdbsearch.drop(columns=['Unnamed: 0','Unnamed: 3'],inplace=True)
imdbsearch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   show_title  863 non-null    object
 1   genres      817 non-null    object
dtypes: object(2)
memory usage: 13.6+ KB


In [43]:
leftupdate = pd.merge(left,imdbsearch, on='show_title' ,how='left')
leftupdate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225760 entries, 0 to 225759
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   country_name                225760 non-null  object
 1   country_iso2                225760 non-null  object
 2   week                        225760 non-null  object
 3   category                    225760 non-null  object
 4   weekly_rank                 225760 non-null  int64 
 5   show_title                  225760 non-null  object
 6   season_title                109197 non-null  object
 7   cumulative_weeks_in_top_10  225760 non-null  int64 
 8   genres_x                    210484 non-null  object
 9   genres_y                    17120 non-null   object
dtypes: int64(2), object(8)
memory usage: 18.9+ MB


In [44]:
nanmask = left['genres'].isna()
left.loc[nanmask,'genres'] = leftupdate.loc[nanmask,'genres_y']
left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225760 entries, 0 to 225759
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   country_name                225760 non-null  object
 1   country_iso2                225760 non-null  object
 2   week                        225760 non-null  object
 3   category                    225760 non-null  object
 4   weekly_rank                 225760 non-null  int64 
 5   show_title                  225760 non-null  object
 6   season_title                109197 non-null  object
 7   cumulative_weeks_in_top_10  225760 non-null  int64 
 8   genres                      225503 non-null  object
dtypes: int64(2), object(7)
memory usage: 25.3+ MB


In [45]:
country = left.groupby('country_name').count()
country['percent'] = 1 - (country['genres']/country['week'])
country.sort_values(by=['percent'],ascending=False)

,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,genres,percent
country_name,,,,,,,,,
South Korea,2420,2420,2420,2420,2420,1115,2420,2271,0.061570
Taiwan,2420,2420,2420,2420,2420,1034,2420,2380,0.016529
Czech Republic,2420,2420,2420,2420,2420,1193,2420,2410,0.004132
Thailand,2420,2420,2420,2420,2420,1054,2420,2411,0.003719
Turkey,2420,2420,2420,2420,2420,1172,2420,2414,0.002479
...,...,...,...,...,...,...,...,...,...
Chile,2420,2420,2420,2420,2420,1185,2420,2420,0.000000
Colombia,2420,2420,2420,2420,2420,1189,2420,2420,0.000000
Jamaica,2420,2420,2420,2420,2420,1188,2420,2420,0.000000


In [46]:
datacountries = left.copy()
datacountries.dropna(subset=['genres'], inplace=True)
datacountries.reset_index(drop=False,inplace=True)
datacountries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225503 entries, 0 to 225502
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   index                       225503 non-null  int64 
 1   country_name                225503 non-null  object
 2   country_iso2                225503 non-null  object
 3   week                        225503 non-null  object
 4   category                    225503 non-null  object
 5   weekly_rank                 225503 non-null  int64 
 6   show_title                  225503 non-null  object
 7   season_title                109015 non-null  object
 8   cumulative_weeks_in_top_10  225503 non-null  int64 
 9   genres                      225503 non-null  object
dtypes: int64(3), object(7)
memory usage: 17.2+ MB


In [47]:
datacountries['genres'].replace({'sci-if':'sci-fi'},inplace=True)
datacountries['genres'] = datacountries['genres'].apply(lambda x:x.strip())
datacountries['genres'] = datacountries['genres'].str.lower()
datacountries['genres'] = datacountries['genres'].str.split(',')
datacountries = datacountries.explode('genres')
datacountries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491021 entries, 0 to 225502
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   index                       491021 non-null  int64 
 1   country_name                491021 non-null  object
 2   country_iso2                491021 non-null  object
 3   week                        491021 non-null  object
 4   category                    491021 non-null  object
 5   weekly_rank                 491021 non-null  int64 
 6   show_title                  491021 non-null  object
 7   season_title                238092 non-null  object
 8   cumulative_weeks_in_top_10  491021 non-null  int64 
 9   genres                      491021 non-null  object
dtypes: int64(3), object(7)
memory usage: 41.2+ MB


In [48]:
genrescount = datacountries.groupby(by=['country_name','genres']).count()
genrescount = genrescount[['index']]
genrescount = genrescount.rename(columns={'index':'count'})
genrescount.to_csv('datacountries.csv')

In [49]:
dataglobal = pd.read_csv('https://github.com/finnjinju/netflix/raw/master/all-weeks-global.csv')
dataglobal['show_title'] = dataglobal['show_title'].str.lower()
globaljoin = pd.merge(dataglobal,imdbNew[['show_title','genres']], on='show_title',how='left')
globaljoin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4840 entries, 0 to 4839
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        4840 non-null   object 
 1   category                    4840 non-null   object 
 2   weekly_rank                 4840 non-null   int64  
 3   show_title                  4840 non-null   object 
 4   season_title                2343 non-null   object 
 5   weekly_hours_viewed         4840 non-null   int64  
 6   runtime                     760 non-null    float64
 7   weekly_views                760 non-null    float64
 8   cumulative_weeks_in_top_10  4840 non-null   int64  
 9   is_staggered_launch         4840 non-null   bool   
 10  episode_launch_details      24 non-null     object 
 11  genres                      4442 non-null   object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 458.5+ KB


In [50]:
nanmaskgb = globaljoin['genres'].isna()
globalremain = globaljoin[nanmaskgb]
globalreleft = pd.merge(globalremain,imdb2[['show_title','genres']], on='show_title', how='left')
globalreleft.set_index(globalremain.index, inplace=True)
globalreleft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 15 to 4832
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        398 non-null    object 
 1   category                    398 non-null    object 
 2   weekly_rank                 398 non-null    int64  
 3   show_title                  398 non-null    object 
 4   season_title                153 non-null    object 
 5   weekly_hours_viewed         398 non-null    int64  
 6   runtime                     36 non-null     float64
 7   weekly_views                36 non-null     float64
 8   cumulative_weeks_in_top_10  398 non-null    int64  
 9   is_staggered_launch         398 non-null    bool   
 10  episode_launch_details      0 non-null      object 
 11  genres_x                    0 non-null      object 
 12  genres_y                    51 non-null     object 
dtypes: bool(1), float64(2), int64(3),

In [51]:
globaljoin.loc[nanmaskgb,'genres'] = globalreleft.loc[nanmaskgb,'genres_y']
globaljoin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4840 entries, 0 to 4839
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        4840 non-null   object 
 1   category                    4840 non-null   object 
 2   weekly_rank                 4840 non-null   int64  
 3   show_title                  4840 non-null   object 
 4   season_title                2343 non-null   object 
 5   weekly_hours_viewed         4840 non-null   int64  
 6   runtime                     760 non-null    float64
 7   weekly_views                760 non-null    float64
 8   cumulative_weeks_in_top_10  4840 non-null   int64  
 9   is_staggered_launch         4840 non-null   bool   
 10  episode_launch_details      24 non-null     object 
 11  genres                      4493 non-null   object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 587.5+ KB


In [52]:
nanmaskgb = globaljoin['genres'].isna()
globalserach = globaljoin[nanmaskgb]
gbsearch = pd.merge(globalserach,imdbsearch, on='show_title', how='left')
gbsearch.set_index(globalserach.index, inplace=True)
gbsearch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347 entries, 15 to 4832
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        347 non-null    object 
 1   category                    347 non-null    object 
 2   weekly_rank                 347 non-null    int64  
 3   show_title                  347 non-null    object 
 4   season_title                131 non-null    object 
 5   weekly_hours_viewed         347 non-null    int64  
 6   runtime                     33 non-null     float64
 7   weekly_views                33 non-null     float64
 8   cumulative_weeks_in_top_10  347 non-null    int64  
 9   is_staggered_launch         347 non-null    bool   
 10  episode_launch_details      0 non-null      object 
 11  genres_x                    0 non-null      object 
 12  genres_y                    343 non-null    object 
dtypes: bool(1), float64(2), int64(3),

In [53]:
globaljoin.loc[nanmaskgb,'genres'] = gbsearch.loc[nanmaskgb,'genres_y']
globaljoin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4840 entries, 0 to 4839
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        4840 non-null   object 
 1   category                    4840 non-null   object 
 2   weekly_rank                 4840 non-null   int64  
 3   show_title                  4840 non-null   object 
 4   season_title                2343 non-null   object 
 5   weekly_hours_viewed         4840 non-null   int64  
 6   runtime                     760 non-null    float64
 7   weekly_views                760 non-null    float64
 8   cumulative_weeks_in_top_10  4840 non-null   int64  
 9   is_staggered_launch         4840 non-null   bool   
 10  episode_launch_details      24 non-null     object 
 11  genres                      4836 non-null   object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 587.5+ KB


In [54]:
dataglobal = globaljoin.copy()
dataglobal.dropna(subset=['genres'], inplace=True)
dataglobal.reset_index(drop=False,inplace=True)
dataglobal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4836 entries, 0 to 4835
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       4836 non-null   int64  
 1   week                        4836 non-null   object 
 2   category                    4836 non-null   object 
 3   weekly_rank                 4836 non-null   int64  
 4   show_title                  4836 non-null   object 
 5   season_title                2343 non-null   object 
 6   weekly_hours_viewed         4836 non-null   int64  
 7   runtime                     759 non-null    float64
 8   weekly_views                759 non-null    float64
 9   cumulative_weeks_in_top_10  4836 non-null   int64  
 10  is_staggered_launch         4836 non-null   bool   
 11  episode_launch_details      24 non-null     object 
 12  genres                      4836 non-null   object 
dtypes: bool(1), float64(2), int64(4),

In [55]:
dataglobal['genres'].replace({'sci-if':'sci-fi'},inplace=True)
dataglobal['genres'] = dataglobal['genres'].apply(lambda x:x.strip())
dataglobal['genres'] = dataglobal['genres'].str.lower()
dataglobal['genres'] = dataglobal['genres'].str.split(',')
dataglobal = dataglobal.explode('genres')
dataglobal.head()
dataglobal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10576 entries, 0 to 4835
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       10576 non-null  int64  
 1   week                        10576 non-null  object 
 2   category                    10576 non-null  object 
 3   weekly_rank                 10576 non-null  int64  
 4   show_title                  10576 non-null  object 
 5   season_title                5086 non-null   object 
 6   weekly_hours_viewed         10576 non-null  int64  
 7   runtime                     1685 non-null   float64
 8   weekly_views                1685 non-null   float64
 9   cumulative_weeks_in_top_10  10576 non-null  int64  
 10  is_staggered_launch         10576 non-null  bool   
 11  episode_launch_details      54 non-null     object 
 12  genres                      10576 non-null  object 
dtypes: bool(1), float64(2), int64(4)

In [56]:
dataglobal.to_csv('dataglobal.csv')

In [57]:
datacountries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491021 entries, 0 to 225502
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   index                       491021 non-null  int64 
 1   country_name                491021 non-null  object
 2   country_iso2                491021 non-null  object
 3   week                        491021 non-null  object
 4   category                    491021 non-null  object
 5   weekly_rank                 491021 non-null  int64 
 6   show_title                  491021 non-null  object
 7   season_title                238092 non-null  object
 8   cumulative_weeks_in_top_10  491021 non-null  int64 
 9   genres                      491021 non-null  object
dtypes: int64(3), object(7)
memory usage: 41.2+ MB


In [58]:
country = datacountries['country_name'].unique()
dfcountry = pd.DataFrame(country,columns=['Country'])
dfcountry

,Country
0,Argentina
1,Australia
2,Austria
3,Bahamas
4,Bahrain
...,...
89,United Kingdom
90,United States
91,Uruguay
92,Venezuela


In [59]:
sub = pd.read_csv('https://github.com/finnjinju/netflix/raw/master/Subscribers.csv',index_col=0)
subnew = pd.merge(dfcountry,sub, on='Country',how='left')
subnew.replace({np.NaN:1},inplace=True)
subnew

,Country,Subscribers,Percentage_cum
0,Argentina,1.0,1.000000
1,Australia,6622078.0,0.709671
2,Austria,961399.0,0.956127
3,Bahamas,1.0,1.000000
4,Bahrain,1.0,1.000000
...,...,...,...
89,United Kingdom,13182309.0,0.502574
90,United States,69548673.0,0.344140
91,Uruguay,1.0,1.000000
92,Venezuela,1.0,1.000000


In [29]:
subnew.to_csv('Subscribers_new.csv')